# 17a: Market Segmentatation

In [1]:
# TEST
import larch.numba as lx
import larch
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 3)
larch._doctest_mode_ = True

/Users/manugallardo/opt/anaconda3/envs/larch-roseta2-dev-env/lib/python3.9/site-packages/larch/numba/model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


For this example, we're going to re-create the market segmentation
versions of model 17 from the
[Self Instructing Manual](http://www.caee.utexas.edu/prof/Bhat/COURSES/LM_Draft_060131Final-060630.pdf). (pp. 133)

Market segmentation can be used to determine whether the impact of other variables is different among population groups. The most common approach to market segmentation is for the analyst to consider sample segments which are mutually exclusive and collectively exhaustive (that is, each case is included in one and only one segment). Models are estimated for the sample associated with each segment and compared to the pooled model (all segments represented by a single model) to determine if there are statistically significant and important differences among the market segments.

In these models, we will segment the market by automobile ownership for households that have one or fewer cars. It is appealing to include a distinct segment for households with no cars since the mode choice behavior of this segment is very different from the rest of the population due to their dependence on non-automobile modes. However, the size of this segment in the dataset is too small, so it is joined with the one car group.

In [2]:
import larch.numba as lx
d = lx.examples.MTC(format='dataset')

We can use the `query_cases` method to create two separate datasets for this work,
and then attach these datasets to two models.

In [3]:
d1 = d.dc.query_cases("numveh <= 1")
d2 = d.dc.query_cases("numveh > 1")
m1 = lx.Model(d1, title="Cars<=1")
m2 = lx.Model(d2, title="Cars>=2")

We will then construct the same model stucture in each, using the 
design worked up for Model 17.

In [4]:
from larch import P, X

for m in (m1,m2):
    m.availability_var = 'avail'
    m.choice_ca_var = 'chose'
    m.utility_ca = (
        + X("totcost/hhinc") * P("costbyincome")
        + X("tottime * (altnum <= 4)") * P("motorized_time")
        + X("tottime * (altnum >= 5)") * P("nonmotorized_time")
        + X("ovtt/dist * (altnum <= 4)") * P("motorized_ovtbydist")
    )
    for a in [4,5,6]:
        m.utility_co[a] += X("hhinc") * P("hhinc#{}".format(a))
    for i in d['alt_names'][1:3]:
        name = str(i.values)
        a = int(i.altid)
        m.utility_co[a] += (
            + X("vehbywrk") * P("vehbywrk_SR")
            + X("wkccbd+wknccbd") * P("wkcbd_"+name)
            + X("wkempden") * P("wkempden_"+name)
            + P("ASC_"+name)
        )
    for i in d['alt_names'][3:]:
        name = str(i.values)
        a = int(i.altid)
        m.utility_co[a] += (
            + X("vehbywrk") * P("vehbywrk_"+name)
            + X("wkccbd+wknccbd") * P("wkcbd_"+name)
            + X("wkempden") * P("wkempden_"+name)
            + P("ASC_"+name)
        ) 
    m.ordering = (
        ('LOS', ".*cost.*", ".*time.*", ".*dist.*",),
        ('Zonal', "wkcbd.*", "wkempden.*",),
        ('Household', "hhinc.*", "vehbywrk.*",),
        ('ASCs', "ASC.*",),
    )

## Independent Estimation

We can estimate these models completely independently.

In [5]:
r1 = m1.maximize_loglike()
r2 = m2.maximize_loglike()

In [9]:
# TEST
from pytest import approx
assert r1.loglike == approx(-1049.2796388550328)
assert r2.loglike == approx(-2296.667143538367)
assert r1.n_cases == 1221
assert r2.n_cases == 3808
assert 'success' in r1.message.lower()
assert 'success' in r2.message.lower()

To have t-statistics and $\rho^2_{\oslash}$ values, we'll also need
to prepare those for each model.

In [10]:
for m in (m1,m2):
    m.calculate_parameter_covariance()
    m.loglike_null()

We can generate a side-by-side summary of the 
two models using the [`joint_parameter_summary`](larch.util.summary.joint_parameter_summary) function.

In [11]:
from larch.util.summary import joint_parameter_summary
joint_parameter_summary([m1,m2])

Cars<=1            Cars>=2        
                                        Param  t-Stat      Param  t-Stat
Category       Parameter                                                
Parameters     ASC_Bike                 0.977    1.39      -3.22   -4.39
               ASC_SR2                  0.593    1.96      -1.98  -15.38
               ASC_SR3+                -0.785   -2.22      -3.72  -20.03
               ASC_Transit               2.26    5.08      -2.16   -5.63
               ASC_Walk                  2.91    5.17      -1.53   -2.67
               costbyincome           -0.0227   -1.64    -0.0981   -6.07
               hhinc#4               -0.00645   -1.81   0.000363    0.14
               hhinc#5                -0.0116   -1.23   -0.00192   -0.30
               hhinc#6                -0.0120   -2.01   0.000670    0.16
               motorized_ovtbydist     -0.113   -4.36     -0.194   -5.88
               motorized_time         -0.0211   -3.49    -0.0187   -3.64
               nonmotorized_time      -0.0440   -5.43    -0.0450   -5.16
               vehbywrk_Bike            -2.66   -4.02     -0.190   -0.58
               vehbywrk_SR              -3.01   -8.64     -0.241   -3.27
               vehbywrk_Transit         -3.96  -10.55     -0.240   -1.77
               vehbywrk_Walk            -3.34   -7.52    -0.0976   -0.46
               wkcbd_Bike               0.396    0.74      0.487    0.97
               wkcbd_SR2                0.372    1.55      0.163    1.09
               wkcbd_SR3+               0.229    0.56       1.33    6.02
               wkcbd_Transit             1.11    4.27       1.28    5.23
               wkcbd_Walk              0.0297    0.08      0.111    0.29
               wkempden_Bike          0.00151    0.80    0.00161    0.96
               wkempden_SR2           0.00204    2.80    0.00107    2.22
               wkempden_SR3+          0.00353    3.89    0.00134    2.45
               wkempden_Transit       0.00316    4.73    0.00288    6.42
               wkempden_Walk          0.00379    3.91  -0.000893   -0.42
----           ----                      ----    ----       ----    ----
Log Likelihood Converged            -1,049.28          -2,296.67        
               Null                 -1,775.42          -5,534.18        
Rho Squared    vs Null                 0.4090             0.5850

## Joint Estimation

Suppose we think completely independent estimation is 
inappropriate, and that while these two market segments behave
differently with respect to some attributes of the choice, they
are also the same on some attributes.  In that case, we'll need
to use joint estimation -- that is, estimating all the parameters
of both models simultaneously.

We'll assume that any parameter that appears in both models with 
the same name is actually the same parameter (with the same value).
As configured so far, that's all the parameters.  But we can edit
one of the models so that the parameter names differ where we
want them to, using `reformat_param`.

Suppose we want different parameters for all the `ca` level of
service variables.  We can change every parameter in the entire
`utility_ca` linear function with a simple formatted replacement.

In [12]:
m2.utility_ca = m2.utility_ca.reformat_param('{}_2Cars')

To change just a selection of parameters, there's a regular
expression approach using different arguments to `reformat_param`. 
Here, we'll replace all the parameters with 'ASC' in them to
be a segment-specific value.

In [13]:
for a in m2.utility_co:
    m2.utility_co[a] = m2.utility_co[a].reformat_param(
        pattern='(ASC)', repl='ASC_2Cars'
    )

Then, we can put our models together into a `ModelGroup` for
joint estimation.

In [14]:
mg = lx.ModelGroup([m1,m2])

In [15]:
# TEST
assert mg.loglike('null') == approx(-7309.600971749634)

The estimation interface for a `ModelGroup` is the 
same as that for a regular model: first `maximize_loglike`,
then `calculate_parameter_covariance` if desired. We can also
use many other Model commands, e.g. `set_cap` to bound our 
parameters to finite values.

In [16]:
mg.set_cap()
rg = mg.maximize_loglike()
mg.calculate_parameter_covariance()

In [17]:
rg

┣                x: ASC_Bike                    -1.393
┃                   ASC_SR2                     -1.582
┃                   ASC_SR3+                    -3.261
┃                   ASC_Transit                 -0.747
┃                   ASC_Walk                     0.314
┃                   costbyincome                -0.017
┃                   hhinc#4                     -0.001
┃                   hhinc#5                     -0.004
┃                   hhinc#6                     -0.002
┃                   motorized_ovtbydist         -0.100
┃                   motorized_time              -0.020
┃                   nonmotorized_time           -0.044
┃                   vehbywrk_Bike               -0.255
┃                   vehbywrk_SR                 -0.314
┃                   vehbywrk_Transit            -0.638
┃                   vehbywrk_Walk               -0.395
┃                   wkcbd_Bike                   0.460
┃                   wkcbd_SR2                    0.233
┃                   wkcbd_SR3+                   1.043
┃                   wkcbd_Transit                1.265
┃                   wkcbd_Walk                   0.100
┃                   wkempden_Bike                0.001
┃                   wkempden_SR2                 0.001
┃                   wkempden_SR3+                0.002
┃                   wkempden_Transit             0.003
┃                   wkempden_Walk                0.002
┃                   ASC_2Cars_Bike              -2.937
┃                   ASC_2Cars_SR2               -1.908
┃                   ASC_2Cars_SR3+              -3.528
┃                   ASC_2Cars_Transit           -1.471
┃                   ASC_2Cars_Walk              -1.005
┃                   costbyincome_2Cars          -0.102
┃                   motorized_ovtbydist_2Cars   -0.187
┃                   motorized_time_2Cars        -0.020
┃                   nonmotorized_time_2Cars     -0.047
┃                   dtype: float64
┣          loglike: -3406.7232214086853
┣        d_loglike: ASC_Bike                     4.531e-04
┃                   ASC_SR2                      9.358e-04
┃                   ASC_SR3+                     1.537e-04
┃                   ASC_Transit                 -4.233e-04
┃                   ASC_Walk                     9.290e-04
┃                   costbyincome                -6.067e-04
┃                   hhinc#4                      6.494e-02
┃                   hhinc#5                      6.001e-02
┃                   hhinc#6                     -5.352e-02
┃                   motorized_ovtbydist         -7.833e-03
┃                   motorized_time               2.096e-02
┃                   nonmotorized_time            6.499e-02
┃                   vehbywrk_Bike                9.046e-04
┃                   vehbywrk_SR                  4.546e-03
┃                   vehbywrk_Transit             1.513e-03
┃                   vehbywrk_Walk               -1.692e-03
┃                   wkcbd_Bike                   4.045e-04
┃                   wkcbd_SR2                    1.106e-03
┃                   wkcbd_SR3+                  -4.495e-04
┃                   wkcbd_Transit               -1.423e-03
┃                   wkcbd_Walk                   3.751e-04
┃                   wkempden_Bike                4.570e-03
┃                   wkempden_SR2                 5.782e-01
┃                   wkempden_SR3+                3.861e-02
┃                   wkempden_Transit            -1.395e+00
┃                   wkempden_Walk                2.783e-01
┃                   ASC_2Cars_Bike               4.876e-04
┃                   ASC_2Cars_SR2                9.621e-04
┃                   ASC_2Cars_SR3+               1.826e-03
┃                   ASC_2Cars_Transit            8.558e-04
┃                   ASC_2Cars_Walk              -1.356e-03
┃                   costbyincome_2Cars           5.548e-03
┃                   motorized_ovtbydist_2Cars    7.841e-03
┃                   motorized_time_2Cars         6.44

In [18]:
# TEST
assert rg.loglike == approx(-3406.7232213364787)
assert rg.n_cases == 5029

To review the estimation results, we can use `ordering`,
`parameter_summary`, and `estimation_statistics` as usual.

In [19]:
mg.ordering = (
    ('LOS', ".*cost.*", ".*time.*", ".*dist.*",),
    ('Zonal', "wkcbd.*", "wkempden.*",),
    ('Household', "hhinc.*", "vehbywrk.*",),
    ('ASCs', "ASC.*",),
)
mg.parameter_summary()

In [20]:
# TEST
import re, platform
mash = lambda x: re.sub('\\s+', ' ', str(x)).strip()
if platform.system() != "Windows":
    assert mash(mg.parameter_summary().data) == mash('''
                                        Value    Std Err  t Stat Signif  Null Value
Category  Parameter                                                                
LOS       costbyincome                -0.0174     0.0117   -1.49                0.0
          costbyincome_2Cars           -0.102     0.0154   -6.60    ***         0.0
          motorized_time              -0.0196    0.00567   -3.46    ***         0.0
          motorized_time_2Cars        -0.0195    0.00496   -3.94    ***         0.0
          nonmotorized_time           -0.0442    0.00776   -5.69    ***         0.0
          nonmotorized_time_2Cars     -0.0471    0.00868   -5.43    ***         0.0
          motorized_ovtbydist          -0.100     0.0237   -4.23    ***         0.0
          motorized_ovtbydist_2Cars    -0.187     0.0309   -6.05    ***         0.0
Zonal     wkcbd_Bike                    0.460      0.364    1.27                0.0
          wkcbd_SR2                     0.233      0.124    1.88                0.0
          wkcbd_SR3+                     1.04      0.192    5.42    ***         0.0
          wkcbd_Transit                  1.27      0.168    7.53    ***         0.0
          wkcbd_Walk                    0.100      0.250    0.40                0.0
          wkempden_Bike               0.00137    0.00124    1.11                0.0
          wkempden_SR2                0.00132   0.000392    3.36    ***         0.0
          wkempden_SR3+               0.00184   0.000461    3.99    ***         0.0
          wkempden_Transit            0.00274   0.000358    7.64    ***         0.0
          wkempden_Walk               0.00248   0.000730    3.39    ***         0.0
Household hhinc#4                    -0.00101    0.00208   -0.48                0.0
          hhinc#5                    -0.00366    0.00527   -0.70                0.0
          hhinc#6                    -0.00232    0.00331   -0.70                0.0
          vehbywrk_Bike                -0.255      0.290   -0.88                0.0
          vehbywrk_SR                  -0.314     0.0740   -4.24    ***         0.0
          vehbywrk_Transit             -0.638      0.132   -4.82    ***         0.0
          vehbywrk_Walk                -0.395      0.198   -1.99      *         0.0
ASCs      ASC_2Cars_Bike                -2.94      0.629   -4.67    ***         0.0
          ASC_2Cars_SR2                 -1.91      0.127  -15.06    ***         0.0
          ASC_2Cars_SR3+                -3.53      0.173  -20.43    ***         0.0
          ASC_2Cars_Transit             -1.47      0.320   -4.60    ***         0.0
          ASC_2Cars_Walk                -1.00      0.517   -1.94                0.0
          ASC_Bike                      -1.39      0.439   -3.17     **         0.0
          ASC_SR2                       -1.58      0.126  -12.57    ***         0.0
          ASC_SR3+                      -3.26      0.206  -15.84    ***         0.0
          ASC_Transit                  -0.747      0.271   -2.76     **         0.0
          ASC_Walk                      0.314      0.416    0.75                0.0
''')

AssertionError: 

In [21]:
mg.estimation_statistics()

<xmle.Elem 'div' with 1 children>